# Imports

In [1]:
import pandas as pd
import torch.nn as nn
import numpy as np
import torch
import random
from torchvision import transforms
import torchvision

# Getting trainsets from google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip './drive/MyDrive/cnn/train.zip'

!unzip './drive/MyDrive/cnn/test_all.zip'

In [4]:
drive.flush_and_unmount()

# Preparations for GPU

In [5]:
torch.cuda.set_device(0)
device = torch.device('cuda')

# Getting test set

In [6]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import os
import natsort
from torch.utils.data import Dataset
from PIL import Image

class TestDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        
        all_imgs = os.listdir(main_dir)
        self._all_images = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self._all_images)

    def __getitem__(self, idx):
        img_name = self._all_images[idx]
        img_path = os.path.join(self.main_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image, img_name

In [8]:
SEED = 42;

transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128
train_to_valid = 0.8

images = torchvision.datasets.ImageFolder("./train/", transform=transform)

valid_index = int(train_to_valid * len(images))

trainset, validset = torch.utils.data.random_split(images, [valid_index, len(images) - valid_index], generator=torch.Generator().manual_seed(SEED))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = TestDataSet("./test_all/", transform=transform)


In [9]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=0)
        self.bn2d1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
        self.bn2d2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0)
        self.bn2d3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=0)
        self.bn2d4 = nn.BatchNorm2d(256)
        
        ## Warstwy max pooling 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(2304, 5000)
        self.bn1 = nn.BatchNorm1d(5000)
        self.d1 = nn.Dropout(0.7)

        self.fc2 = nn.Linear(5000, 5000)
        self.bn2 = nn.BatchNorm1d(5000)
        self.d2 = nn.Dropout(0.7)

        self.fc3 = nn.Linear(5000, 50)

    def forward(self, x):
        x = self.bn2d1(F.relu(self.conv1(x)))
        x = self.pool1(x)

        x = self.bn2d2(F.relu(self.conv2(x)))

        x = self.bn2d3(F.relu(self.conv3(x)))
        x = self.pool2(x)

        x = self.bn2d4(F.relu(self.conv4(x)))
        x = self.pool3(x)
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = self.bn1(F.relu(self.fc1(x)))
        x = self.d1(x)
        
        x = self.bn2(F.relu(self.fc2(x)))
        x = self.d2(x)
        
        x = self.fc3(x)
        return x

In [10]:
import torch.optim as optim

net = Net()
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00005)


In [11]:
net.train()

EPOCHS = 15

for epoch in range(EPOCHS): 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('[%d/%d] loss: %.3f' %
          (epoch+1 , EPOCHS,  running_loss / 2000))
    running_loss = 0.0

print('Finished Training')

[1/15] loss: 0.914
[2/15] loss: 0.766
[3/15] loss: 0.697
[4/15] loss: 0.640
[5/15] loss: 0.597
[6/15] loss: 0.559
[7/15] loss: 0.529
[8/15] loss: 0.493
[9/15] loss: 0.463
[10/15] loss: 0.432
[11/15] loss: 0.406
[12/15] loss: 0.372
[13/15] loss: 0.341
[14/15] loss: 0.314
[15/15] loss: 0.283
Finished Training


## Testing on train set

In [12]:
net.eval()

correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # calculate outputs by running images through the network 
        outputs = net(images)
        # the class with the highest energy is what we choose as "prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 86 %


## Testing on validation set

In [13]:
net.eval()

correct_pred = {classname: 0 for classname in range(50)}
total_pred = {classname: 0 for classname in range(50)}

correct = 0
total = 0
with torch.no_grad():
    for data in validloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # calculate outputs by running images through the network 
        outputs = net(images)
        # the class with the highest energy is what we choose as "prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[label.item()] += 1
            total_pred[label.item()] += 1

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:d} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy of the network on the test images: 51 %
Accuracy for class 0 is: 39.2 %
Accuracy for class 1 is: 48.5 %
Accuracy for class 2 is: 40.6 %
Accuracy for class 3 is: 43.3 %
Accuracy for class 4 is: 44.4 %
Accuracy for class 5 is: 56.0 %
Accuracy for class 6 is: 60.5 %
Accuracy for class 7 is: 45.8 %
Accuracy for class 8 is: 33.8 %
Accuracy for class 9 is: 50.0 %
Accuracy for class 10 is: 36.8 %
Accuracy for class 11 is: 66.5 %
Accuracy for class 12 is: 61.6 %
Accuracy for class 13 is: 28.4 %
Accuracy for class 14 is: 59.4 %
Accuracy for class 15 is: 46.5 %
Accuracy for class 16 is: 33.2 %
Accuracy for class 17 is: 54.4 %
Accuracy for class 18 is: 41.6 %
Accuracy for class 19 is: 32.6 %
Accuracy for class 20 is: 69.7 %
Accuracy for class 21 is: 56.1 %
Accuracy for class 22 is: 83.7 %
Accuracy for class 23 is: 47.9 %
Accuracy for class 24 is: 61.1 %
Accuracy for class 25 is: 43.2 %
Accuracy for class 26 is: 75.4 %
Accuracy for class 27 is: 43.5 %
Accuracy for class 28 is: 47.8 %
Accu

## Predicting test_all data

In [14]:
net.eval()

all_preds = []

with torch.no_grad():
  for data in testset:
    image, image_name = data
    image = image.to(device)
    image = image.unsqueeze(dim = 0)
    output = net(image).squeeze()
    _, predicted = torch.max(output.data, 0)
    all_preds.append([image_name, predicted.item()])

pd.DataFrame(all_preds).to_csv("results.csv", index=False, header=None)